# Render animations in AMASS format in Maya

This tutorial shows how to render animations in [Maya](https://www.autodesk.com/products/maya/overview) for [AMASS](https://amass.is.tue.mpg.de/) dataset collection

## Download dataset from AMASS collection

- Download dataset: [AMASS](https://amass.is.tue.mpg.de/) is a large database of human motion unifying different optical marker-based motion capture datasets by representing them within a common framework and parameterization. After opening an [AMASS account](https://amass.is.tue.mpg.de/login.php), you may download the specific dataset you want from [AMASS Download](https://amass.is.tue.mpg.de/download.php), and download the `Dody Data` in the `SMPL+H` column, for example.

<div>
<img src="../../../../../dataset/Amass/imgs/download_npy.png" width="500"/>
</div>

- Download FBX skeleton: Download the FBX models for `SMPL+H` from [MANO downloads](https://mano.is.tue.mpg.de/downloads) after creating a [MANO account](https://mano.is.tue.mpg.de/en/sign_in)

<div>
<img src="../../../../../dataset/Amass/imgs/download_fbx.png" width="500"/>
</div>


In [8]:
# after downloading the dataset, unzip the tar.gz file, and assign the path for `npy` relative to the\
# the root folder of `GenMotion/src`
amass_npz_fname = '../thirdParty/accad/C2 - Run to stand_poses.npz' 

## Open Maya with socket server

- After starting Maya, open the [Script Editor](https://knowledge.autodesk.com/support/maya/learn-explore/caas/CloudHelp/cloudhelp/2020/ENU/Maya-Scripting/files/GUID-7C861047-C7E0-4780-ACB5-752CD22AB02E-htm.html#:~:text=To%20open%20the%20Script%20Editor,bottom%20right%20corner%20of%20Maya.)

<div>
<img src="../../../../images/maya_script_editor.png" width="500"/>
</div>

- Start a new Python script in the `Script Editor` and type the following command to open a [commandport] as a a local [**socket server**](https://download.autodesk.com/us/maya/2009help/CommandsPython/commandPort.html) with a **PORT** code e.g. 12345

```python
import maya.cmds as cmds

# Open a command port with the default name "mayaCommand".
cmds.commandPort(n="localhost:12345")
```

## Import FBX model into Maya

In Maya scene, import the downloaded `SMPL+H` fbx file;
(or you may refer to [Meshcapade Wiki](https://meshcapade.wiki/) for more information with the `SMPL` format)

<div>
<img src="../../../../images/maya_import_fbx.png" width="500"/>
</div>

Now everything is ready to make animation from scripts in Maya. 

<div>
<img src="../../../../images/maya_import_finished.png" width="500"/>
</div>

In Maya scene view, you will have see the `skeleton prefix` (e.g. *f_avg*) in the **Outliner** view, the `FBX` with skin and skeleton in **View Port**, and the `socket server` opened by the **Script Editor**. 

## Animation

Now we are ready to make the animation from Python scripts.

In [2]:
# change working directory to `src` folder in the root folder
import os
os.chdir("../../../../../src")
os.getcwd()

'e:\\researches\\GenMotion\\src'

In [18]:
# import packages
import numpy as np
from tqdm.auto import tqdm

# import GenMotion modules

from dataset.amass_params import SMPL_H_SKELETON # recognize the skeleton type as SMPL_H
from render.maya.utils import MayaController

In [9]:
# load body data from npz file
bdata = np.load(amass_npz_fname)

In [12]:
# Set up Maya Controller
mc = MayaController()

In [14]:
## Test Maya command by setting the key frame at timeline
# mc.SetCurrentTimeFrame(10)

## Have a look at the body data information
# print('Data keys available:%s'%list(bdata.keys()))

In [16]:
# Get the frame rate of from the body data
print("framerate:", bdata['mocap_framerate'])

framerate: 120.0


In [17]:
# Set one frame for the animation
def SetOneFrame(frame, body_data, joints_info_list: list, joint_prefix = "f_avg_"):
    '''Set one maya frame from load data
    
    :param frame: int
    :param body_data: array
    :param joints_info_list: list
    :return:
    '''
    #  set current time frame
    mc.SetCurrentTimeFrame(frame)

    for i in range(0,len(joints_info_list)):
        # get rotation from body data
        rotateX, rotateY, rotateZ = np.rad2deg(body_data['poses'][frame][i*3:(i+1)*3])
        
        # get joint name
        joint_name = joint_prefix + joints_info_list[i]
        
        # if i == 0, set root translation
        if i == 0:
            joint_name = joint_prefix + "root"
            translateX,translateY,translateZ = body_data['trans'][frame]
            mc.SetObjectAttribute(joint_name, "translateX", translateX)
            mc.SetObjectAttribute(joint_name, "translateY", translateY)
            mc.SetObjectAttribute(joint_name, "translateZ", translateZ)
            
        # set rotation for joints
        mc.SetObjectAttribute(joint_name, "rotateX", rotateX)
        mc.SetObjectAttribute(joint_name, "rotateY", rotateY)
        mc.SetObjectAttribute(joint_name, "rotateZ", rotateZ)
        
        mc.SetCurrentKeyFrameForPositionAndRotation(joint_name)
        #break

In [19]:
# Now we maker animation for every 10 frame
frame_interval = 10
for frame in tqdm(range(0, len(bdata["poses"]), frame_interval)):
    SetOneFrame(frame, bdata, SMPL_H_SKELETON)

100%|██████████| 64/64 [00:42<00:00,  1.52it/s]


Now we are able to see the animation in Maya!

<div>
<img src="../../../../images/maya_amass_demo1.gif" width="500"/>
</div>